In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet

seed = 2021
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 모델 클래스 정의 (CustomEfficientNet)
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name="efficientnet-b0", num_classes=6, freeze_backbone=False):
        super(CustomEfficientNet, self).__init__()
        # Pretrained EfficientNet Backbone
        self.backbone = EfficientNet.from_pretrained(model_name)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        in_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Identity()
        
        # CNN 계층 2개로 구성된 추가 블록
        self.extra_conv = nn.Sequential(
            nn.Conv2d(in_channels=in_features, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # 분류기
        self.classifier = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone.extract_features(x)  # EfficientNet backbone
        x = self.extra_conv(x)                 # CNN 블록 (2개의 Conv2D 계층 포함)
        x = x.view(x.size(0), -1)              # Flatten
        x = self.classifier(x)                 # Fully Connected Layers
        return x


# 하이퍼파라미터
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 & 로더
image_size = EfficientNet.get_image_size(model_name)
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 및 손실 함수, 최적화기
model = CustomEfficientNet(model_name=model_name, num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 및 평가 함수
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# 학습 루프
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


Loaded pretrained weights for efficientnet-b0
[Epoch 1/50] Train Loss: 1.3178 | Train Acc: 0.5056
  Test Loss: 3.9495  | Test Acc: 0.3446
[Epoch 2/50] Train Loss: 0.7273 | Train Acc: 0.7825
  Test Loss: 1.1729  | Test Acc: 0.7175
[Epoch 3/50] Train Loss: 0.4606 | Train Acc: 0.8729
  Test Loss: 0.8968  | Test Acc: 0.7797
[Epoch 4/50] Train Loss: 0.3461 | Train Acc: 0.9054
  Test Loss: 1.3795  | Test Acc: 0.7853
[Epoch 5/50] Train Loss: 0.5503 | Train Acc: 0.8828
  Test Loss: 1.1810  | Test Acc: 0.7627
[Epoch 6/50] Train Loss: 0.3087 | Train Acc: 0.9096
  Test Loss: 1.1094  | Test Acc: 0.8305
[Epoch 7/50] Train Loss: 0.2948 | Train Acc: 0.9266
  Test Loss: 1.3127  | Test Acc: 0.7910
[Epoch 8/50] Train Loss: 0.2737 | Train Acc: 0.9280
  Test Loss: 1.4429  | Test Acc: 0.8192
[Epoch 9/50] Train Loss: 0.2012 | Train Acc: 0.9435
  Test Loss: 1.0985  | Test Acc: 0.8418
[Epoch 10/50] Train Loss: 0.2070 | Train Acc: 0.9477
  Test Loss: 1.0889  | Test Acc: 0.7910
[Epoch 11/50] Train Loss: 0.2086 